In [82]:
from pathlib import Path
import torch, numpy as np, pandas as pd
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
import timm

In [31]:
device = torch.device("cpu")

In [72]:
data_path = Path() / 'paddy-disease-classification' / 'train_images'
data = ImageFolder(data_path, transforms.Compose([
    v2.Resize((480, 480)),
    transforms.ToTensor(),
    ]))

In [81]:
train_data, valid_data = random_split(data, [0.8, 0.2])

train_dls = DataLoader(train_data, batch_size=64)
valid_dls = DataLoader(valid_data, batch_size=64)

In [71]:
model = timm.create_model('resnet26d')

In [84]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [86]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # back-propagate
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")

In [87]:
def valid_loop(dataloader, model, loss_fn):
    model.eval()
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100 * correct):>0.1f}, Avg Loss: {test_loss:>8f} \n")


In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train_loop(train_dls, model, loss_fn, optimizer)
    valid_loop(valid_dls, model, loss_fn)
print("Done!")

In [89]:
torch.save(model, "paddies-model.pth")